In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [2]:
from govrfp.rfpinsights._chromadb import return_collection, add_entries_to_collection
path = "tests/mar-14/data/"
critical_collection = return_collection(path=path, collection_name="critical")
informative_collection = return_collection(path=path, collection_name="informative")

In [4]:
critical_data_path = "tests/mar-14/02-critical-compliances.txt"
informative_data_path = "tests/mar-14/02-informative-compliances.txt"
# open both and read all lines separately
with open(critical_data_path, 'r') as file:
    critical_data = file.readlines()
with open(informative_data_path, 'r') as file:
    informative_data = file.readlines()
type(critical_data), type(informative_data), len(critical_data), len(informative_data)

(list, list, 115, 188)

In [5]:
add_entries_to_collection(
    docs=critical_data,
    metadata=[{"id": str(i)} for i in range(len(critical_data))],
    ids = [str(i) for i in range(len(critical_data))],
    collection=critical_collection
)
add_entries_to_collection(
    docs=informative_data,
    metadata=[{"id": str(i)} for i in range(len(informative_data))],
    ids = [str(i) for i in range(len(informative_data))],
    collection=informative_collection
)
critical_collection.count(), informative_collection.count()

(115, 188)

In [7]:
test_data_path = "tests/mar-14/RFP_Factors_and_Rating_Criteria_-_Phase 2_.txt"
with open(test_data_path, 'r') as file:
    test_data = file.readlines()
test_data = [line.strip() for line in test_data]
test_data = [line for line in test_data if line]
test_data[:5]

["Strength - is an attribute of the Offeror's proposal that exceeds the specified performance or capability requirements in a way that is beneficial to the government.",
 'Weakness - is a flaw in the proposal that increases the risk of unsuccessful contract performance',
 'Significant Weakness - in the proposal is a flaw that appreciably increases the risk of unsuccessful contract performance.',
 'Deficiency - is a material failure of a proposal to meet a Government requirement or a combination of significant weaknesses in a proposal that increases the risk of unsuccessful contract performance to an unacceptable level',
 'A proposal with a deficiency cannot be found eligible for award and is rated “unacceptable”. Proposals rated that have 1 or more deficiencies for any of the Factors are NOT eligible for award and will be excluded from further consideration.']

In [8]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [10]:
PROMPT_TEMPLATE = (
    "From the given CRITICAL compliances, and INFORMATIVE compliances, return 1 for CRITICAL and 0 for INFORMATIVE for each of the user query based on what type of compliance it is."
    "Return None if the user query isn't found in either of the compliances."
    "CRITICAL compliances:\n"
    "{critical}\n"
    "INFORMATIVE compliances:\n"
    "{informative}\n"
    "User Query: {query}\n"
    "Return 0, 1 or None based on the type of compliance the user query is.\n"
    "Response:\n"
)
print(PROMPT_TEMPLATE)

From the given CRITICAL compliances, and INFORMATIVE compliances, return 1 for CRITICAL and 0 for INFORMATIVE for each of the user query based on what type of compliance it is.Return None if the user query isn't found in either of the compliances.CRITICAL compliances:
{critical}
INFORMATIVE compliances:
{informative}
User Query: {query}
Return 0, 1 or None based on the type of compliance the user query is.
Response:



In [13]:
def get_top_n(query, collection, top_n= 15):
    results = collection.query(query_texts=[query], n_results=top_n)
    docs = results['documents'][0]
    return "\n".join([f"{i+1}. {doc}" for i, doc in enumerate(docs)])

In [15]:
import tqdm
ratings = []
for query in tqdm.tqdm(test_data):
    prompt = PROMPT_TEMPLATE.format(
        critical=get_top_n(query, critical_collection),
        informative=get_top_n(query, informative_collection),
        query=query
    )
    response = llm.complete(prompt).text
    ratings.append(response)

100%|██████████| 48/48 [01:42<00:00,  2.13s/it]


In [20]:
# count the number of none
none_count = sum([1 for rating in ratings if 'none' in rating.lower()])
none_count

0

In [23]:
# count of 1
one_count = sum([1 for rating in ratings if '1' in rating])
one_count

38

In [24]:
# count of 0
zero_count = sum([1 for rating in ratings if '0' in rating])
zero_count

11